In [11]:
import numpy as np
import os.path as osp
from collections import defaultdict
from sklearn.neighbors import NearestNeighbors  # Students: you can use this implementation to find the 
                                                # Nearest-Neigbors

In [12]:
# Load golden distances (pairwise matrix, or corresponding model/part names in golden_names)
golden_part_dist_file = '../data/golden_dists.npz'
golden_data = np.load(golden_part_dist_file, allow_pickle=True)
golden_part_dist = golden_data['golden_part_dist']
golden_names = golden_data['golden_names']
print(len(golden_names))  # models-name/part combinations
print(golden_names[0])

484
c59cdd1537bd75ddd0818327fc390a5__2__


In [13]:
# Load/organize golden part-aware distances.
sn_id_to_parts = defaultdict(list)
id_to_part_loc = dict()

for i, name in enumerate(golden_names):
    # Extract shape-net model ids of golden, map them to their parts.
    sn_id, _, part_id, _, _ = name.split('_')
    sn_id_to_parts[sn_id].append(part_id)
    
    # Map shape-net model id and part_id to location in distance matrix, (the order is the same).
    id_to_part_loc[(sn_id, part_id)] = i

In [42]:
out_n = 1024

for model_tag in [f'pc_ae_out{out_n}_', f'part_pc_ae_out{out_n}_', f'experiment_']:
    ae_emb_file = f'../data/out/{model_tag}_latent_codes.npz'
    in_d = np.load(ae_emb_file)

    latent_codes = in_d['latent_codes']
    test_names = in_d['test_names']

    # TODO: Use golden distances and matchings to solve question (g)
    nn = NearestNeighbors(n_neighbors=2)
    nn.fit(latent_codes)

    encoding_distances = np.zeros(len(test_names))
    num_shared_parts = np.zeros(len(test_names))
    latent_distances = np.empty(len(test_names))
    for i, sn_name in enumerate(test_names):
        parts_of_model = set(sn_id_to_parts[sn_name])

        nn_distances, nn_indices = nn.kneighbors([latent_codes[i]],  return_distance=True)
        latent_distances[i] = nn_distances[0, 1]
        matched_neighbor = test_names[nn_indices[0, 1]] # Students find the model's name of the Nearest-Neighbor
        parts_of_neighbor = set(sn_id_to_parts[matched_neighbor])

        # compute the requested distances.
        # Use id_to_part_loc for each model/part combination

        parts_in_both = parts_of_model.intersection(parts_of_neighbor)
        for k in parts_in_both:
            encoding_distances[i] += golden_part_dist[id_to_part_loc[(sn_name, k)], id_to_part_loc[(matched_neighbor, k)]]
            num_shared_parts[i] += 1

        parts_only_A = parts_of_model.difference(parts_of_neighbor)
        
        for k in parts_only_A:
            encoding_distances[i] += max(
                [golden_part_dist[id_to_part_loc[(sn_name, k)], id_to_part_loc[(matched_neighbor, u)]] for u in parts_of_neighbor] 
            )

    print(f'''
        {model_tag} PART AWARENESS: \n
        Cumulative encoding distance = {encoding_distances.sum():.4f} 
        Avg. shared parts = {num_shared_parts.mean():.2f} 
        Avg. latent distance = {latent_distances.mean():.2f}    
    ''')


        pc_ae_out1024_ PART AWARENESS: 

        Cumulative encoding distance = 406.0312 
        Avg. shared parts = 3.15 
        Avg. latent distance = 0.21    
    

        part_pc_ae_out1024_ PART AWARENESS: 

        Cumulative encoding distance = 403.0605 
        Avg. shared parts = 3.19 
        Avg. latent distance = 0.30    
    

        experiment_ PART AWARENESS: 

        Cumulative encoding distance = 426.0120 
        Avg. shared parts = 3.17 
        Avg. latent distance = 0.20    
    
